In [ ]:
#Importar librerias
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import validation_curve
import matplotlib.pyplot as plt 

In [ ]:
#leer el dataframe 
dataframe=pd.read_csv('/content/95-amk-343.csv')
dataframe = dataframe.drop(['id', 'aislamiento', 'AMKI'], axis='columns')

In [ ]:
#defino las variables "X" y "y"
X = dataframe.drop(['AMIKACINA'], axis='columns') #e
print(X)
print(y)

     APH(3')-VIa  AAC(6')-Ian  ...  OXA-171  OXA-69
0           -1.0          1.0  ...     -1.0    -1.0
1           -1.0          1.0  ...     -1.0    -1.0
2           -1.0          1.0  ...     -1.0    -1.0
3           -1.0          1.0  ...     -1.0    -1.0
4           -1.0         -1.0  ...     -1.0    -1.0
..           ...          ...  ...      ...     ...
337         -1.0         -1.0  ...     -1.0    -1.0
338         -1.0          1.0  ...     -1.0    -1.0
339         -1.0          1.0  ...     -1.0    -1.0
340         -1.0         -1.0  ...     -1.0    -1.0
341         -1.0         -1.0  ...     -1.0    -1.0

[342 rows x 58 columns]
0      128
1       64
2       32
3       32
4       32
      ... 
337     32
338     64
339     64
340     16
341     64
Name: AMIKACINA, Length: 342, dtype: int64


In [ ]:
#division de los datos en entrenamiento y prueba 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=27)
#print(X_train)
print(y_test)

91     64
199    32
21     64
215     4
230    32
       ..
86     32
142     8
246    32
200    32
173    32
Name: AMIKACINA, Length: 69, dtype: int64


In [ ]:
#convierto la variable "y" en log 
y_train1 = np.log2(y_train)
y_test1 = np.log2(y_test)
print(y_test1)
print(y_train1)

91     6.0
199    5.0
21     6.0
215    2.0
230    5.0
      ... 
86     5.0
142    3.0
246    5.0
200    5.0
173    5.0
Name: AMIKACINA, Length: 69, dtype: float64
69     5.0
330    4.0
282    4.0
213    5.0
147    5.0
      ... 
312    6.0
31     7.0
328    4.0
184    5.0
19     7.0
Name: AMIKACINA, Length: 273, dtype: float64


In [ ]:
modelo = GradientBoostingRegressor()

In [ ]:
#ajuste de hiperparametros por gridesearch
modelo1 = GradientBoostingRegressor()
max_depth = [5, 6, 7, 8, 10]
min_samples_split = [2, 3, 5, 6] 
max_features = ['auto', 'sqrt', 0.4, 0.5, 0.2, 0.30, 0.6]
min_samples_leaf = [1, 2, 3]
learning_rate = [0.001, 0.01, 0.05, 0.1, 0.2] 
n_estimators = [40, 50, 60, 70, 80, 100, 200, 300 ]

#hyperF = dict(learning_rate = learning_rate, n_estimators = n_estimators)
hyperF = dict(max_depth = max_depth, learning_rate = learning_rate, min_samples_leaf = min_samples_leaf, min_samples_split = min_samples_split, max_features = max_features)
gridF = GridSearchCV(modelo1, hyperF, cv = 10, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(X_train, y_train)
print('Best Params: ', gridF.best_params_)



Fitting 10 folds for each of 2100 candidates, totalling 21000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 392 tasks      | elapsed:   19.9s
[Parallel(n_jobs=-1)]: Done 1392 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 2792 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 4592 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 6792 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done 9392 tasks      | elapsed:  8.1min
[Parallel(n_jobs=-1)]: Done 12392 tasks      | elapsed: 10.8min
[Parallel(n_jobs=-1)]: Done 15792 tasks      | elapsed: 13.6min
[Parallel(n_jobs=-1)]: Done 19592 tasks      | elapsed: 16.9min


Best Params:  {'learning_rate': 0.01, 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 3, 'min_samples_split': 3}


[Parallel(n_jobs=-1)]: Done 21000 out of 21000 | elapsed: 18.3min finished


In [ ]:
#definir el modelo con los valores optimos establecidos con gride search 
modelo = GradientBoostingRegressor(max_depth = 10, min_samples_leaf = 3, min_samples_split = 3, max_features = 'sqrt', learning_rate = 0.01)

In [ ]:
#tunning de hiperparametros (resultados con RMSE)
#parametros = [1, 3, 8, 10, 20, 30, 50, 100, 150, 200, 250] # parametros para n_estimators
#train_scores, valid_scores = validation_curve(GradientBoostingRegressor(max_depth= 8, learning_rate = 0.01, max_features= 'sqrt', min_samples_leaf= 3, min_samples_split = 6), X = X_train, y=y_train1, param_name ="n_estimators", param_range= parametros, cv=5, scoring="neg_mean_squared_error")
#parametros = [1, 2, 3, 4, 5, 6, 7, 10, 15, 20]  #parametros para max_depth
#train_scores, valid_scores = validation_curve(GradientBoostingRegressor(), X = X_train, y=y_train1, param_name ="max_depth", param_range= parametros, cv=5, scoring="neg_mean_squared_error")
#parametros = [1, 2, 3, 4, 5, 6, 7, 8, 10] #parametros para min_samples-split y min_samples_leaf
#train_scores, valid_scores = validation_curve(GradientBoostingRegressor(), X = X_train, y=y_train1, param_name ="min_samples_leaf", param_range= parametros, cv=5, scoring="neg_mean_squared_error")
parametros = [0.001, 0.005, 0.01, 0.05, 0.1, 0.2, 0.5] #parametros para learning_rate
train_scores, valid_scores = validation_curve(GradientBoostingRegressor(), X = X_train, y=y_train1, param_name ="learning_rate", param_range= parametros, cv=5, scoring="neg_mean_squared_error")
train_scores_positive = np.array(train_scores) * -1
valid_scores_positive = np.array(valid_scores) * -1
train_mean = np.mean(train_scores_positive, axis=1)
train_std = np.std(train_scores_positive, axis=1)
val_mean = np.mean(valid_scores_positive, axis=1)
val_std = np.std(valid_scores_positive, axis=1)
print('Score train: ', train_scores_positive)
print('Score test: ', valid_scores_positive)
print('RMSE train: ', train_mean)
print('RMSE val: ', val_mean)
#grafica de validacion RMSE
plt.plot(parametros, train_mean, color='blue', marker='o', markersize=5, label='training score')
plt.fill_between(parametros, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')

plt.plot(parametros, val_mean, color='green', linestyle='--', marker='s', markersize=5, label='validation score')
plt.fill_between(parametros, val_mean + val_std, val_mean - val_std, alpha=0.15, color='green')

plt.grid()
plt.xscale('log')
#plt.xscale('linear')
#plt.xticks(parametros)
plt.title('Amikacina 100%-GBR')
plt.legend(loc='upper righ')
plt.xlabel('Tasa de aprendizaje')
plt.ylabel('RMSE')
plt.ylim([-1, 5])
plt.show()

In [ ]:
#entreno modelo y realiazo predicciones 
modelo.fit(X_train, y_train1) #entreno
y_pred = modelo.predict(X_test)
#hago predicciones 
print(y_pred)
y_pred1 = 2**np.round(modelo.predict(X_test))
print(y_pred1)
verdaderos = y_test.to_numpy()
print(verdaderos)

[4.23712547 4.77867561 5.13172579 3.66156324 4.30293034 3.36422594
 3.77493237 4.56747917 4.8203424  5.13172579 3.88237831 4.63414282
 3.76150903 4.49717303 4.83429524 4.63414282 4.99425759 3.59125244
 5.29128156 4.28621467 3.11296167 4.77867561 4.22532027 5.32356519
 3.62020594 3.563658   4.75275907 3.40869611 5.13172579 4.65788081
 4.73486347 3.38287507 4.63554554 4.20530636 5.67900879 3.29300689
 3.58303685 4.63414282 4.71132799 5.25353929 5.21362262 5.09373361
 3.75727763 4.63414282 4.63751117 4.74129155 3.96816565 3.57694441
 5.0441856  4.56166659 4.74129155 5.15283491 4.54437445 3.60753235
 4.69331073 5.67900879 4.78817543 4.69331073 3.63046514 4.74129155
 3.60555219 3.88831567 4.79614257 3.57735738 4.73469459 4.68662955
 4.63414282 4.77867561 4.56747917]
[16. 32. 32. 16. 16.  8. 16. 32. 32. 32. 16. 32. 16. 16. 32. 32. 32. 16.
 32. 16.  8. 32. 16. 32. 16. 16. 32.  8. 32. 32. 32.  8. 32. 16. 64.  8.
 16. 32. 32. 32. 32. 32. 16. 32. 32. 32. 16. 16. 32. 32. 32. 32. 32. 16.
 32. 64. 

In [ ]:
#obtengo los valores que muestran el desempeño del modelo
print('Mean Absolute Error_test:', metrics.mean_absolute_error(y_test1, y_pred))
print('Mean Absolute Error_train:', metrics.mean_absolute_error(y_train1, modelo.predict(X_train)))
print('Mean Squared Error_test:', metrics.mean_squared_error(y_test1, y_pred))
print('Mean Squared Error_train:', metrics.mean_squared_error(y_train1, modelo.predict(X_train)))
print('Root Mean Squared Error_test:', np.sqrt(metrics.mean_squared_error(y_test1, y_pred)))
print('RMSE_train:', np.sqrt(metrics.mean_squared_error(y_train1, modelo.predict(X_train))))
print('r2_train:', metrics.r2_score(y_train1, modelo.predict(X_train)))
print('r2_teste:', metrics.r2_score(y_test1, (y_pred)))

Mean Absolute Error_test: 0.7983176385423434
Mean Absolute Error_train: 0.7673813002589419
Mean Squared Error_test: 0.9600001450861448
Mean Squared Error_train: 0.9923025506714878
Root Mean Squared Error_test: 0.9797959711522317
RMSE_train: 0.996143840352129
r2_train: 0.5849031409319766
r2_teste: 0.46241346850680587


In [ ]:
#funcion para calcular la excatitud con +/- una dilucion 
def within_dillution(y_test, y_pred1, dill=2):
    frac = np.array(y_test) / np.array(y_pred1)
    return ((frac >= 1/dill) & (frac <= dill)).mean()

In [ ]:
#saco accuracy con +/- 1 dilucion, VME y ME
acc2 = within_dillution(y_test, y_pred1, dill=2)
acc4 = within_dillution(y_test, y_pred1, dill=4)
threshold = 16 
threshold1 = 32 #colacel el punto de corte para el antibiotico que esta probando
predDF = pd.DataFrame({'y_pred1':y_pred1, 'y_test':y_test})
vme = len(predDF[(predDF.y_pred1<=threshold) & (predDF.y_test>threshold1)])
me = len(predDF[(predDF.y_pred1>threshold1) & (predDF.y_test<=threshold)])
print(vme)
print(me)
print(acc2)
print(acc4)

2
0
0.8405797101449275
1.0
